In [3]:
# Here is a simple program that displays the camera feed in a cv2.namedWindow and will save images inside bounding box when 'c' is pressed on keyboard. 
# It will also quit if you hit ESC.

import cv2
import os
import time
from PIL import Image
from keras.models import load_model, model_from_json
from keras.preprocessing import image
import numpy as np

# dimensions of our images
image_x, image_y = 224, 224

cam = cv2.VideoCapture(1)
if cam.read()[0]==False:
    cam = cv2.VideoCapture(0)
#Prediction Classes
gesture_list = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','OK GOOGLE','P','Q','R','S','SPACE','T','U','V','W','X','Y','Z']

def get_bounding_box(img):
    x, y, w, h = 420, 140, 200, 200
    cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 3)
    
def get_cropped_image(img):
    x, y, w, h = 420, 140, 200, 200
#     cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 3)
    imgCrop = img[y:y+h, x:x+w]
    return imgCrop

def get_thres(img):
    # convert to grayscale
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     applying gaussian blur
    value = (5, 5)
    blurred = cv2.GaussianBlur(grey, value, 0)

#     thresholdin: Otsu's Binarization method
    _, thresh = cv2.threshold(blurred, 127, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return thresh

def start_camera():
    frames=0
    sentence = ""
    while True:
        img = cam.read()[1]
        img = cv2.flip(img, 1)
        img_cropped = get_cropped_image(img)
        thresh = get_thres(img_cropped)
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        contour = max(contours, key = cv2.contourArea)
        img_name= "gesture.jpg"
        if len(contours) > 0:
            if cv2.contourArea(contour) < 10000 and frames > 50:
                try:
                    cv2.imwrite(img_name, thresh)
                except Exception as e:
                    pred = "--"
                    print("Something Happened :",e)
                    break
        else:
            print("No Hand Gesture recognized")
        get_bounding_box(img)
        cv2.putText(thresh, str(cv2.contourArea(contour)), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 0.7, (127, 127, 255))
        cv2.imshow("Thresh",thresh)
        cv2.imshow("Capturing gesture", img)
        keypress = cv2.waitKey(1)
        if not keypress==-1:
            print("Key number: ",keypress)
        if keypress%256 == 27:
            # 'ESC' pressed
            print("Escape hit, closing...")
            break
        frames +=1
try:
    start_camera()
except Exception as e:
    print("Something Happened :",e)
cam.release()
cv2.destroyAllWindows()

Key number:  27
Escape hit, closing...
